CASE STUDY 1
CONTRIBUTOR: MURAD ISMAYILZADA UMCS

In [ ]:
# pip install pandas matplotlib seaborn scikit-learn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 600)

: 

In [40]:
cust = pd.read_csv("casestudy1_MuradIsmayilzadaCustomer.csv") #customer details
trans = pd.read_csv("casestudy1_MuradIsmayilzadaTransaction.csv") #transaction

In [ ]:
trans.head()

In [ ]:
cust.head()

In [ ]:
cust.info(),trans.info()

In [ ]:
# Merge the dataframes on the 'customerEmail' column
merged_df = pd.merge(cust, trans, on='customerEmail', how='inner')

# Verify the merged dataframe
merged_df.head()

In [ ]:
# Remove the specified columns
merged_df = merged_df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'])
merged_df.head()

EDA On the Merged _DF
1.Basic Information about the dataset

In [ ]:
# Display the first few rows of the dataframe
print("First few rows of the dataframe:")
print(merged_df.head())

# Get basic information about the dataframe
print("\nSummary of the dataframe:")
print(merged_df.info())

# Summary statistics of numerical columns
print("\nSummary statistics of numerical columns:")
print(merged_df.describe())

# Check for missing values
print("\nMissing values in the dataframe:")
print(merged_df.isnull().sum())

2. Data Distribution and Visualization:


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
numerical_features = ['No_Transactions', 'No_Orders', 'No_Payments']
# Histograms for numerical features
plt.figure(figsize=(10, 6))
merged_df[numerical_features].hist(bins=20)
plt.suptitle('Histograms of Numerical Features')
plt.show()

In [ ]:
# Boxplots for numerical features
sns.boxplot(data=merged_df[numerical_features])
plt.title('Boxplots of Numerical Features')
plt.show()

As there are Outliers in the No_Payments and No_Orders columns, Removing is the best solution for proper further analysis

In [ ]:
# Calculate the first quartile (Q1) and third quartile (Q3)
Q1 = merged_df[numerical_features].quantile(0.25)
Q3 = merged_df[numerical_features].quantile(0.75)

# Calculate the interquartile range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = (merged_df[numerical_features] < lower_bound) | (merged_df[numerical_features] > upper_bound)

# Remove outliers
merged_df_cleaned = merged_df[~outliers.any(axis=1)]
merged_df_cleaned.shape

Given that the outliers constitute only about 3.3% of the total data (27 out of 819 rows), and without a clear understanding of their origin or impact, I lean towards removing them. Removing outliers in this scenario seems reasonable as it's a relatively small portion of the dataset and their presence may potentially distort the analysis or model performance. However, it's important to document this decision and consider the potential implications of removing outliers on the overall analysis.


In [ ]:
# Calculate the first quartile (Q1) and third quartile (Q3)
Q1 = merged_df[numerical_features].quantile(0.25)
Q3 = merged_df[numerical_features].quantile(0.75)

# Calculate the interquartile range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = (merged_df[numerical_features] < lower_bound) | (merged_df[numerical_features] > upper_bound)

# Get rows containing outliers
outlier_rows = merged_df[outliers.any(axis=1)]


outlier_rows.head()

In [ ]:
additional_categorical_features = ['paymentMethodRegistrationFailure', 'paymentMethodType', 'paymentMethodProvider', 'transactionFailed', 'orderState']

# Store the numerical counts for each category in a dictionary
numerical_counts = {}
for feature in additional_categorical_features:
    counts = merged_df_cleaned[feature].value_counts()
    numerical_counts[feature] = counts

# Display numerical counts
for feature, counts in numerical_counts.items():
    print(f'Numerical counts for {feature}:')
    print(counts)
    print()

In [ ]:
additional_categorical_features = ['paymentMethodRegistrationFailure', 'paymentMethodType', 'paymentMethodProvider', 'transactionFailed', 'orderState']

for feature in additional_categorical_features:
    plt.figure(figsize=(16, 6))
    ax = sns.countplot(data=merged_df_cleaned, x=feature)

    # Calculate percentages for each category
    total_count = len(merged_df_cleaned)
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height() / total_count)
        x = p.get_x() + p.get_width() / 2
        y = p.get_height()
        ax.annotate(percentage, (x, y), ha='center', va='bottom')

    plt.title(f'Count Plot of {feature}')
    plt.show()

somthing 

In [ ]:
merged_df_cleaned


3. Correlation Analysis

In [ ]:
# Correlation matrix
correlation_matrix = merged_df_cleaned[["No_Transactions","No_Orders","No_Payments",
                                        "paymentMethodRegistrationFailure",
                                        "Fraud","transactionAmount","transactionFailed"]].corr()

# Heatmap of correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True)
plt.title('Correlation Matrix')
plt.show()

Transaction Metrics and Fraud Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot for No_Transactions
sns.countplot(data=merged_df_cleaned, x='No_Transactions', hue='Fraud', ax=axes[0])
axes[0].set_title('No_Transactions vs Fraud')
axes[0].set_xlabel('No_Transactions')
axes[0].set_ylabel('Count')

# Plot for No_Orders
sns.countplot(data=merged_df_cleaned, x='No_Orders', hue='Fraud', ax=axes[1])
axes[1].set_title('No_Orders vs Fraud')
axes[1].set_xlabel('No_Orders')
axes[1].set_ylabel('Count')

# Plot for No_Payments
sns.countplot(data=merged_df_cleaned, x='No_Payments', hue='Fraud', ax=axes[2])
axes[2].set_title('No_Payments vs Fraud')
axes[2].set_xlabel('No_Payments')
axes[2].set_ylabel('Count')

plt.tight_layout()
plt.show()

Feature Engineering

In [ ]:
merged_df_cleaned['Transaction_Success_Rate'] = merged_df_cleaned['No_Transactions'] / (merged_df_cleaned['No_Transactions'] + merged_df_cleaned['transactionFailed'])

#Reasoning: This feature captures the proportion of successful transactions for each customer,
#providing insights into their transaction success rate.

2. Order Frequency


In [57]:
#merged_df_cleaned['Order_Frequency'] = merged_df_cleaned['No_Orders'] / merged_df_cleaned['No_Transactions']
#Reasoning: Order frequency gives an idea of how often a customer places orders relative to the number of transactions 
# they make. 
# It could indicate whether a customer tends to make multiple orders per transaction or vice versa.
merged_df_cleaned = merged_df_cleaned.copy()
merged_df_cleaned['Order_Frequency'] = merged_df_cleaned['No_Orders'] / merged_df_cleaned['No_Transactions']


Historical behavior

In [58]:
merged_df_cleaned['Past_Fraud_Transactions'] = merged_df_cleaned['Fraud'].cumsum().shift(fill_value=0)
#Reasoning:It provides insight into a customer's historical involvement in fraudulent activities.

Feature importance

In [ ]:
# Fill NaN values with 0
data = merged_df_cleaned.fillna(0)

pd.set_option('display.max_columns', None)

data.columns

In [ ]:
data.select_dtypes(include=['object']).columns.tolist()


In [ ]:
#cat_cols contains the names of categorical columns
cat_cols = ['customerEmail',
 'customerPhone',
 'customerDevice',
 'customerIPAddress',
 'customerBillingAddress',
 'transactionId',
 'orderId',
 'paymentMethodId',
 'paymentMethodType',
 'paymentMethodProvider',
 'orderState'] 

# Create a copy of the DataFrame to avoid modifying the original data
data_encoded = data.copy()

# Iterate over each categorical column
for col in cat_cols:
    # Calculate the frequency of each category
    freq = data_encoded[col].value_counts(normalize=True)
    
    # Map the frequency values to the original categories
    data_encoded[col + '_freq_encoded'] = data_encoded[col].map(freq)

# Drop the original categorical columns if needed
data_encoded.drop(cat_cols, axis=1, inplace=True)
data_encoded.columns

In [ ]:
# NOTE   Transaction_TotalAmount HAS BEEN DELETED bE CAREFUL
# Importing libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


df_selected = data_encoded[['No_Transactions', 'No_Orders', 'No_Payments', 'Fraud',
       'paymentMethodRegistrationFailure', 'transactionAmount',
       'transactionFailed', 'Transaction_Success_Rate', 'Order_Frequency',
       'Past_Fraud_Transactions',
       'customerEmail_freq_encoded', 'customerPhone_freq_encoded',
       'customerDevice_freq_encoded', 'customerIPAddress_freq_encoded',
       'customerBillingAddress_freq_encoded', 'transactionId_freq_encoded',
       'orderId_freq_encoded', 'paymentMethodId_freq_encoded',
       'paymentMethodType_freq_encoded', 'paymentMethodProvider_freq_encoded',
       'orderState_freq_encoded']]


# Split the data into features (X) and target variable (y)
X = df_selected.drop('Fraud', axis=1)
y = df_selected['Fraud']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a Random Forest Classifier model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Feature Importances
feature_importances = model.feature_importances_
feature_names = X.columns

# Create a DataFrame to store feature names and their importance scores
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importance DataFrame
print(feature_importance_df)

Trying RFE

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier


X = data_encoded.drop(columns=['Fraud'])
y = data_encoded['Fraud']

# Initialize a random forest classifier 
clf = RandomForestClassifier()

# Initialize RFE with the classifier and the number of features to select
rfe = RFE(estimator=clf, n_features_to_select=10)

# Fit RFE to the data
rfe.fit(X, y)

# Get the selected features
selected_features = pd.DataFrame({'Feature': X.columns, 'Selected': rfe.support_})

# Print the selected features
print(selected_features[selected_features['Selected']])

Model  Building

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Select the features
selected_features = [
    'No_Orders', 'No_Payments', 'transactionAmount', 'Past_Fraud_Transactions',
     'customerEmail_freq_encoded', 'customerDevice_freq_encoded',
    'customerIPAddress_freq_encoded', 'customerBillingAddress_freq_encoded',
    'paymentMethodId_freq_encoded'
]

# Prepare the feature matrix X and target vector y
X = data_encoded[selected_features]
y = data_encoded['Fraud']  # Assuming 'Fraud' is the target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Model Evaluation (Accuracy=0.94)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

In [ ]:
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

True Negative (TN): The model correctly predicted 68 instances as negative.
False Positive (FP): The model incorrectly predicted 2 instances as positive when they were actually negative.
False Negative (FN): The model incorrectly predicted 9 instances as negative when they were actually positive.
True Positive (TP): The model correctly predicted 80 nstances as positive.

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

In [ ]:
# Extract feature importances from the model
feature_importances = rf_classifier.feature_importances_

# Plot the feature importances
plt.figure(figsize=(10, 8))
plt.barh(selected_features, feature_importances)
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importances')
plt.show()